# Kirisu 2

#### Simple editor for stopped-flow and other spectra

by *Sergio Boneta Martínez*  
based on the original **kirisu** by *Jose Ramon Peregrina* and *Jorge Estrada*

GPLv3 (C) 2022-2025 @ *Universidad de Zaragoza*

In [ ]:
#@title Prepare the environment
#@markdown Execute this cell before running the following cells. Only need to be done once, the rest can be run multiple times.
%%capture

#@markdown Plots are nice and useful, but araise as source of errors and can be time comsuming for batch working. Uncheck the *plot* option to disable them.
plot = True  # @param {type:"boolean"}
debug = False

%pip install git+https://github.com/unizar-flav/kirisu2.git

import os
import zipfile
from copy import deepcopy

import bokeh.io

from tdspectrum import TDSpectrum

try:
    from google.colab import files, widgets
    GOOGLE_COLAB = True
except ImportError:
    GOOGLE_COLAB = False

bokeh.io.output_notebook()


In [ ]:
#@title Upload spectra
#@markdown Upload one or several files from your local computer containing multiple spectra. Alternatively, a zip file containing multiple spectra can be uploaded.
#@markdown The format to be readed can be manually specified or automatically inferred from the extension of the file. If the format is not recognized, 'glb' will be assumed.

format = ""  # @param ["", "glb", "csv", "bk3a"]


if GOOGLE_COLAB:
    uploaded = files.upload()
else:
    uploaded = {}
    uploaded['test.glb'] = open('test.glb', 'rb').read()

spectrum = TDSpectrum()
spectra = []
spectra_t = []
if uploaded:
    for filename, filestr in uploaded.items():
        if zipfile.is_zipfile(filename):
            with zipfile.ZipFile(filename) as zf:
                for zi in zf.infolist():
                    if zi.file_size == 0:
                        raise ValueError(f"Zip file {filename} is empty")
                    with zf.open(zi) as f:
                        filestr = f.read().decode('cp1252')
                        try:
                            spectrum.read(zi.filename, filestr, format=format)
                            spectra.append(deepcopy(spectrum))
                        except:
                            print(f"ERROR: '{zi.filename}' could not be readed")
                            if debug:
                                raise
        else:
            try:
                spectrum.read(filename, filestr.decode('cp1252'), format=format)
                spectra.append(deepcopy(spectrum))
            except:
                print(f"ERROR: '{filename}' could not be readed")
                if debug:
                    raise

for spectrum in spectra:
    if plot:
        spectrum.plot_tabs()
    print(spectrum)


In [ ]:
#@title Process spectra
#@markdown Modify the spectra according to the input values provided. This cell can be run as many times as needed until a satisfactory plot is reached. If multiple spectra have been uploaded, the same modifications are applied to all of them.

#@markdown Smooth the absorbance with respect the wavelenght for each time.
smooth = 'No'  # @param ["No", "gaussian - Gaussian Filter", "sma - Simple Moving Average", "median - Median Filter"]
smooth_level = 1  #@param {type:"slider", min:0.2, max:3, step:0.1}

#@markdown Modify the spectra to make all times zero at a specific wavelength. Not performed if unchecked.
zero = True  # @param {type:"boolean"}
lambda_zero = 1000  # @param {type:"number"}

#@markdown Trim the spectra to only include up to a minimum and maximum time and wavelenght. Any value out of range will not be modified.
time_min = 0  # @param {type:"number"}
time_max = 1000  # @param {type:"number"}
lambda_min = 0  # @param {type:"number"}
lambda_max = 1000  # @param {type:"number"}

# check number of spectra
if len(spectra) == 0:
    raise ValueError("No spectra to process. Please, upload a file in the cell above.")
# process spectra
smooth = smooth.split()[0]
spectra_t = [deepcopy(spectrum) for spectrum in spectra]
for spectrum in spectra_t:
    if zero:
        spectrum.zero(lambda_zero)
    if smooth != 'No':
        spectrum.smooth(smooth, smooth_level)
    spectrum.trim([time_min, time_max], [lambda_min, lambda_max])
    if plot:
        spectrum.plot_tabs()
    print(spectrum)


In [ ]:
#@title Download processed spectra
#@markdown Write the modified spectra to a new file and download it to your computer. The uploaded name with a '_t' will be used. If the format is left blank, the original formatting will be used (or '.glb' by default).

#@markdown If 'zip' is checked, all the spectra will be zipped and downloaded in a single file named "spectra_kirisu2.zip". Otherwise downloaded individually.

format = ""  # @param ["", "glb", "csv", "bk3a"]
zip = False  # @param {type:"boolean"}
file_name_zip = "spectra_kirisu2.zip"

# check number of spectra
if len(spectra_t) == 0:
    raise ValueError("No spectra to download. Please, process a file in the cell above.")
# write spectra to file
file_names = []
for spectrum in spectra_t:
    filename_trim = os.path.splitext(spectrum.filename_trim)
    file_name = f"{filename_trim[0]}.{format or filename_trim[1][1:] or 'glb'}"
    file_names.append(file_name)
    spectrum.write(file_names[-1])
# zip files if requested and download
if zip:
    print("Zipping...")
    with zipfile.ZipFile(file_name_zip, 'w') as zip_file:
        for name in file_names:
            zip_file.write(name)
    files.download(file_name_zip)
else:
    for name in file_names:
        files.download(name)
